In [4]:
import pywt
import os
import pandas as pd
import librosa
import librosa.display
import glob 
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
import seaborn as sns
from fitter import Fitter, get_common_distributions, get_distributions

from sklearn.preprocessing import StandardScaler
import sklearn

from utils import plot_projections

from scipy.stats import kurtosis, skew

%matplotlib widget

In [5]:
laser_dir = '/home/hashim/PHD/audio_data/Laser_Original_New/Laser/'
laser_files = list(glob.glob(os.path.join(laser_dir, '*.mp3')))
laser_files.sort()
print(laser_files)
print(len(laser_files))

['/home/hashim/PHD/audio_data/Laser_Original_New/Laser/laser_000.mp3', '/home/hashim/PHD/audio_data/Laser_Original_New/Laser/laser_001.mp3', '/home/hashim/PHD/audio_data/Laser_Original_New/Laser/laser_002.mp3', '/home/hashim/PHD/audio_data/Laser_Original_New/Laser/laser_003.mp3', '/home/hashim/PHD/audio_data/Laser_Original_New/Laser/laser_004.mp3', '/home/hashim/PHD/audio_data/Laser_Original_New/Laser/laser_005.mp3', '/home/hashim/PHD/audio_data/Laser_Original_New/Laser/laser_007.mp3', '/home/hashim/PHD/audio_data/Laser_Original_New/Laser/laser_008.mp3', '/home/hashim/PHD/audio_data/Laser_Original_New/Laser/laser_009.mp3', '/home/hashim/PHD/audio_data/Laser_Original_New/Laser/laser_010.mp3']
10


In [6]:
orig_dir = '/home/hashim/PHD/audio_data/Laser_Original_New/Original/'
orig_files = list(glob.glob(os.path.join(orig_dir, '*.mp3')))
orig_files.sort()
print(orig_files)
print(len(orig_files))

['/home/hashim/PHD/audio_data/Laser_Original_New/Original/original_000.mp3', '/home/hashim/PHD/audio_data/Laser_Original_New/Original/original_001.mp3', '/home/hashim/PHD/audio_data/Laser_Original_New/Original/original_002.mp3', '/home/hashim/PHD/audio_data/Laser_Original_New/Original/original_003.mp3', '/home/hashim/PHD/audio_data/Laser_Original_New/Original/original_004.mp3', '/home/hashim/PHD/audio_data/Laser_Original_New/Original/original_005.mp3', '/home/hashim/PHD/audio_data/Laser_Original_New/Original/original_006.mp3', '/home/hashim/PHD/audio_data/Laser_Original_New/Original/original_007.mp3', '/home/hashim/PHD/audio_data/Laser_Original_New/Original/original_008.mp3', '/home/hashim/PHD/audio_data/Laser_Original_New/Original/original_009.mp3']
10


In [7]:
db1 = pywt.Wavelet('db1')

## Laser

In [8]:
%%capture
# compute coefficients for both laser and then compute variance, skewness and kurtosis

laser_sigma_feat = []
laser_skew_feat = []
laser_kurt_feat = []

for lf in laser_files:
    audio_data, sr = librosa.load(lf, res_type='kaiser_fast')
#     stft = librosa.stft(audio_data, n_fft=256)
#     S_db = librosa.amplitude_to_db(np.abs(stft), ref=np.max)
    coeffs = pywt.wavedec(audio_data, db1, mode='constant', level=5)
    
    
    sigma_feat = []
    skew_feat = []
    kurt_feat = []
    
    for cf in coeffs:
        
#         cf = cf - np.mean(cf)
        
        sigma_feat.append(np.var(cf))
        skew_feat.append(skew(cf))
        kurt_feat.append(kurtosis(cf))
        
    laser_sigma_feat.append(sigma_feat)
    laser_skew_feat.append(skew_feat)
    laser_kurt_feat.append(kurt_feat)    
    
laser_sigma_feat = np.array(laser_sigma_feat)
laser_skew_feat = np.array(laser_skew_feat)
laser_kurt_feat = np.array(laser_kurt_feat)

In [9]:
print(laser_sigma_feat.shape)
print(laser_skew_feat.shape)
print(laser_kurt_feat.shape)

(10, 6)
(10, 6)
(10, 6)


## Original

In [10]:
%%capture
# compute coefficients for both laser and then compute variance, skewness and kurtosis

orig_sigma_feat = []
orig_skew_feat = []
orig_kurt_feat = []

for of in orig_files:
    audio_data, sr = librosa.load(of, res_type='kaiser_fast')
#     stft = librosa.stft(audio_data, n_fft=256)
#     S_db = librosa.amplitude_to_db(np.abs(stft), ref=np.max)
    coeffs = pywt.wavedec(audio_data, db1, mode='constant', level=5)
    
    
    sigma_feat = []
    skew_feat = []
    kurt_feat = []
    
    for cf in coeffs:
        
#         cf = cf - np.mean(cf)
        
        sigma_feat.append(np.var(cf))
        skew_feat.append(skew(cf))
        kurt_feat.append(kurtosis(cf))
        
    orig_sigma_feat.append(sigma_feat)
    orig_skew_feat.append(skew_feat)
    orig_kurt_feat.append(kurt_feat)    
    
orig_sigma_feat = np.array(orig_sigma_feat)
orig_skew_feat = np.array(orig_skew_feat)
orig_kurt_feat = np.array(orig_kurt_feat)

In [11]:
print(orig_sigma_feat.shape)
print(orig_skew_feat.shape)
print(orig_kurt_feat.shape)

(10, 6)
(10, 6)
(10, 6)


In [12]:
# cluster plotting function for sigma, skew and kurtosis

def plot_clusters_skk(band_num):
    
    plt.figure(figsize=(10,3))
    
#     sigma vs skew
    plt.subplot(1, 3, 1)
    plt.scatter(orig_sigma_feat[:,band_num], orig_skew_feat[:,band_num], marker='o', label="Original")
    plt.scatter(laser_sigma_feat[:,band_num], laser_skew_feat[:,band_num], marker='x', label="Laser")
    plt.xlabel("variance")
    plt.ylabel("skew")
    plt.legend()
    
#     skew vs kurtosis
    plt.subplot(1, 3, 2)
    plt.scatter(orig_skew_feat[:,band_num], orig_kurt_feat[:,band_num], marker='o', label="Original")
    plt.scatter(laser_skew_feat[:,band_num], laser_kurt_feat[:,band_num], marker='x', label="Laser")
    plt.xlabel("skew")
    plt.ylabel("kurtosis")
    plt.legend()
     
#     kurtosis vs sigma
    plt.subplot(1, 3, 3)
    plt.scatter(orig_kurt_feat[:,band_num], orig_sigma_feat[:,band_num], marker='o', label="Original")
    plt.scatter(laser_kurt_feat[:,band_num], laser_sigma_feat[:,band_num], marker='x', label="Laser")
    plt.xlabel("kurtosis")
    plt.ylabel("variance")
    plt.legend()
    
    plt.suptitle("Laser vs Original for {}th Band".format(band_num))
    
    fig = plt.figure(figsize=(12,6))
#     plt.figure(figsize=(20,6))
    ax = fig.add_subplot(projection='3d')
    ax.scatter(orig_sigma_feat[:,band_num], orig_skew_feat[:,band_num], orig_kurt_feat[:,band_num], marker='o', label="Original")
    ax.scatter(laser_sigma_feat[:,band_num], laser_skew_feat[:,band_num], laser_kurt_feat[:,band_num], marker='x', label="Laser")
    ax.set_xlabel('variance')
    ax.set_ylabel('skew')
    ax.set_zlabel('kurtosis')
    ax.legend()

In [13]:
aprox_detail_coeff = 0 
plot_clusters_skk(aprox_detail_coeff)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [11]:
aprox_detail_coeff = 1 
plot_clusters_skk(aprox_detail_coeff)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [12]:
aprox_detail_coeff = 2 
plot_clusters_skk(aprox_detail_coeff)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [13]:
aprox_detail_coeff = 3 
plot_clusters_skk(aprox_detail_coeff)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [14]:
aprox_detail_coeff = 4 
plot_clusters_skk(aprox_detail_coeff)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [15]:
aprox_detail_coeff = 5 
plot_clusters_skk(aprox_detail_coeff)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Below Results are for the VAD audios 

In [16]:
laser_dir = '/home/hashim/PHD/audio_data/Laser_Original_New/Laser_cleaned/'
laser_files_vad = list(glob.glob(os.path.join(laser_dir, '*.mp3')))
laser_files_vad.sort()
# print(laser_files)
print(len(laser_files_vad))

72


In [17]:
orig_dir = '/home/hashim/PHD/audio_data/Laser_Original_New/Original_cleaned/'
orig_files_vad = list(glob.glob(os.path.join(orig_dir, '*.mp3')))
orig_files_vad.sort()
# print(orig_files)
print(len(orig_files_vad))

74


## Laser

In [18]:
%%capture
# compute coefficients for both laser and then compute variance, skewness and kurtosis

laser_sigma_feat = []
laser_skew_feat = []
laser_kurt_feat = []

for lf in laser_files_vad:
    audio_data, sr = librosa.load(lf, res_type='kaiser_fast')
#     stft = librosa.stft(audio_data, n_fft=256)
#     S_db = librosa.amplitude_to_db(np.abs(stft), ref=np.max)
    coeffs = pywt.wavedec(audio_data, db1, mode='constant', level=5)
    
    
    sigma_feat = []
    skew_feat = []
    kurt_feat = []
    
    for cf in coeffs:
        
        cf = cf - np.mean(cf)
        
        sigma_feat.append(np.var(cf))
        skew_feat.append(skew(cf))
        kurt_feat.append(kurtosis(cf))
        
    laser_sigma_feat.append(sigma_feat)
    laser_skew_feat.append(skew_feat)
    laser_kurt_feat.append(kurt_feat)    
    
laser_sigma_feat = np.array(laser_sigma_feat)
laser_skew_feat = np.array(laser_skew_feat)
laser_kurt_feat = np.array(laser_kurt_feat)

In [19]:
print(laser_sigma_feat.shape)
print(laser_skew_feat.shape)
print(laser_kurt_feat.shape)

(72, 6)
(72, 6)
(72, 6)


## Original

In [20]:
%%capture
# compute coefficients for both laser and then compute variance, skewness and kurtosis

orig_sigma_feat = []
orig_skew_feat = []
orig_kurt_feat = []

for of in orig_files_vad:
    audio_data, sr = librosa.load(of, res_type='kaiser_fast')
#     stft = librosa.stft(audio_data, n_fft=256)
#     S_db = librosa.amplitude_to_db(np.abs(stft), ref=np.max)
    coeffs = pywt.wavedec(audio_data, db1, mode='constant', level=5)
    
    
    sigma_feat = []
    skew_feat = []
    kurt_feat = []
    
    for cf in coeffs:
        
        cf = cf - np.mean(cf)
        
        sigma_feat.append(np.var(cf))
        skew_feat.append(skew(cf))
        kurt_feat.append(kurtosis(cf))
        
    orig_sigma_feat.append(sigma_feat)
    orig_skew_feat.append(skew_feat)
    orig_kurt_feat.append(kurt_feat)    
    
orig_sigma_feat = np.array(orig_sigma_feat)
orig_skew_feat = np.array(orig_skew_feat)
orig_kurt_feat = np.array(orig_kurt_feat)

In [21]:
print(orig_sigma_feat.shape)
print(orig_skew_feat.shape)
print(orig_kurt_feat.shape)

(74, 6)
(74, 6)
(74, 6)


In [22]:
aprox_detail_coeff = 0 
plot_clusters_skk(aprox_detail_coeff)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [23]:
aprox_detail_coeff = 1 
plot_clusters_skk(aprox_detail_coeff)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [24]:
aprox_detail_coeff = 2 
plot_clusters_skk(aprox_detail_coeff)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [25]:
aprox_detail_coeff = 3 
plot_clusters_skk(aprox_detail_coeff)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [26]:
aprox_detail_coeff = 4 
plot_clusters_skk(aprox_detail_coeff)

/tmp/ipykernel_17744/2478823229.py:5: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  plt.figure(figsize=(10,3))


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [27]:
aprox_detail_coeff = 5 
plot_clusters_skk(aprox_detail_coeff)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# SVM with Linear Kernel

In [28]:
import sklearn

from sklearn.svm import LinearSVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import plot_confusion_matrix

In [29]:
X_laser = np.concatenate((laser_sigma_feat, laser_skew_feat, laser_kurt_feat), axis=1)
laser_label = np.array(['laser']*X_laser.shape[0])
print(laser_label)
print(laser_label.shape)
X_laser.shape

['laser' 'laser' 'laser' 'laser' 'laser' 'laser' 'laser' 'laser' 'laser'
 'laser' 'laser' 'laser' 'laser' 'laser' 'laser' 'laser' 'laser' 'laser'
 'laser' 'laser' 'laser' 'laser' 'laser' 'laser' 'laser' 'laser' 'laser'
 'laser' 'laser' 'laser' 'laser' 'laser' 'laser' 'laser' 'laser' 'laser'
 'laser' 'laser' 'laser' 'laser' 'laser' 'laser' 'laser' 'laser' 'laser'
 'laser' 'laser' 'laser' 'laser' 'laser' 'laser' 'laser' 'laser' 'laser'
 'laser' 'laser' 'laser' 'laser' 'laser' 'laser' 'laser' 'laser' 'laser'
 'laser' 'laser' 'laser' 'laser' 'laser' 'laser' 'laser' 'laser' 'laser']
(72,)


(72, 18)

In [30]:
X_orig = np.concatenate((orig_sigma_feat, orig_skew_feat, orig_kurt_feat), axis=1)
orig_label = np.array(['original']*X_orig.shape[0])
print(orig_label)
print(orig_label.shape)
X_orig.shape

['original' 'original' 'original' 'original' 'original' 'original'
 'original' 'original' 'original' 'original' 'original' 'original'
 'original' 'original' 'original' 'original' 'original' 'original'
 'original' 'original' 'original' 'original' 'original' 'original'
 'original' 'original' 'original' 'original' 'original' 'original'
 'original' 'original' 'original' 'original' 'original' 'original'
 'original' 'original' 'original' 'original' 'original' 'original'
 'original' 'original' 'original' 'original' 'original' 'original'
 'original' 'original' 'original' 'original' 'original' 'original'
 'original' 'original' 'original' 'original' 'original' 'original'
 'original' 'original' 'original' 'original' 'original' 'original'
 'original' 'original' 'original' 'original' 'original' 'original'
 'original' 'original']
(74,)


(74, 18)

In [31]:
clf = make_pipeline(StandardScaler(), LinearSVC())

In [32]:
X = np.concatenate((X_laser, X_orig))
X.shape

(146, 18)

In [33]:
y = np.concatenate((laser_label, orig_label))
y.shape

(146,)

In [34]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [35]:
X_train.shape

(97, 18)

In [36]:
clf.fit(X_train, y_train)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('linearsvc', LinearSVC())])

In [37]:
y_pred = clf.predict(X_test)

In [38]:
print("accuracy = ", accuracy_score(y_test, y_pred))

accuracy =  0.9591836734693877


In [39]:
cm = confusion_matrix(y_test, y_pred)

In [40]:
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()

In [41]:
(tn, fp, fn, tp)

(24, 2, 0, 23)

In [42]:
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=clf.classes_)
disp.plot()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [43]:
clf_svc = make_pipeline(StandardScaler(), SVC(gamma='auto'))

In [44]:
clf_svc.fit(X_train, y_train)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('svc', SVC(gamma='auto'))])

In [45]:
y_pred_svc = clf_svc.predict(X_test)

In [46]:
print("accuracy = ", accuracy_score(y_test, y_pred_svc))

accuracy =  0.9795918367346939


In [47]:
cm_svc = confusion_matrix(y_test, y_pred_svc)

In [48]:
tn, fp, fn, tp = cm_svc.ravel()

In [49]:
(tn, fp, fn, tp)

(25, 1, 0, 23)

In [50]:
disp = ConfusionMatrixDisplay(confusion_matrix=cm_svc, display_labels=clf_svc.classes_)
disp.plot()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# with Log sigma

In [51]:
import math

In [52]:
# in_array = [0, 0.2, 0.4, 0.6, 0.8, 1]
# out_array = np.log(in_array)
# out_array1 = math.log10(in_array)
# print(out_array1)
# out_array

In [53]:
%%capture
# compute coefficients for both laser and then compute variance, skewness and kurtosis

laser_sigma_feat = []
laser_skew_feat = []
laser_kurt_feat = []

for lf in laser_files_vad:
    audio_data, sr = librosa.load(lf, res_type='kaiser_fast')
#     stft = librosa.stft(audio_data, n_fft=256)
#     S_db = librosa.amplitude_to_db(np.abs(stft), ref=np.max)
    coeffs = pywt.wavedec(audio_data, db1, mode='constant', level=5)
    
    
    sigma_feat = []
    skew_feat = []
    kurt_feat = []
    
    for cf in coeffs:
        
#         cf = cf
        
        sigma_feat.append(np.log(np.var(cf)))
        skew_feat.append(skew(cf))
        kurt_feat.append(kurtosis(cf))
        
    laser_sigma_feat.append(sigma_feat)
    laser_skew_feat.append(skew_feat)
    laser_kurt_feat.append(kurt_feat)    
    
laser_sigma_feat = np.array(laser_sigma_feat)
laser_skew_feat = np.array(laser_skew_feat)
laser_kurt_feat = np.array(laser_kurt_feat)

In [54]:
print(laser_sigma_feat.shape)
print(laser_skew_feat.shape)
print(laser_kurt_feat.shape)

(72, 6)
(72, 6)
(72, 6)


In [55]:
%%capture
# compute coefficients for both laser and then compute variance, skewness and kurtosis

orig_sigma_feat = []
orig_skew_feat = []
orig_kurt_feat = []

for of in orig_files_vad:
    audio_data, sr = librosa.load(of, res_type='kaiser_fast')
#     stft = librosa.stft(audio_data, n_fft=256)
#     S_db = librosa.amplitude_to_db(np.abs(stft), ref=np.max)
    coeffs = pywt.wavedec(audio_data, db1, mode='constant', level=5)
    
    
    sigma_feat = []
    skew_feat = []
    kurt_feat = []
    
    for cf in coeffs:
        
#         cf = cf
        
        sigma_feat.append(np.log(np.var(cf)))
        skew_feat.append(skew(cf))
        kurt_feat.append(kurtosis(cf))
        
    orig_sigma_feat.append(sigma_feat)
    orig_skew_feat.append(skew_feat)
    orig_kurt_feat.append(kurt_feat)    
    
orig_sigma_feat = np.array(orig_sigma_feat)
orig_skew_feat = np.array(orig_skew_feat)
orig_kurt_feat = np.array(orig_kurt_feat)

In [56]:
aprox_detail_coeff = 0 
plot_clusters_skk(aprox_detail_coeff)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [57]:
aprox_detail_coeff = 1 
plot_clusters_skk(aprox_detail_coeff)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [58]:
aprox_detail_coeff = 2 
plot_clusters_skk(aprox_detail_coeff)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [59]:
aprox_detail_coeff = 3 
plot_clusters_skk(aprox_detail_coeff)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [60]:
aprox_detail_coeff = 4 
plot_clusters_skk(aprox_detail_coeff)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [61]:
aprox_detail_coeff = 5 
plot_clusters_skk(aprox_detail_coeff)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# SVM for log sigma data

In [62]:
X_laser = np.concatenate((laser_sigma_feat, laser_skew_feat, laser_kurt_feat), axis=1)
laser_label = np.array(['laser']*X_laser.shape[0])
print(laser_label)
print(laser_label.shape)
X_laser.shape

['laser' 'laser' 'laser' 'laser' 'laser' 'laser' 'laser' 'laser' 'laser'
 'laser' 'laser' 'laser' 'laser' 'laser' 'laser' 'laser' 'laser' 'laser'
 'laser' 'laser' 'laser' 'laser' 'laser' 'laser' 'laser' 'laser' 'laser'
 'laser' 'laser' 'laser' 'laser' 'laser' 'laser' 'laser' 'laser' 'laser'
 'laser' 'laser' 'laser' 'laser' 'laser' 'laser' 'laser' 'laser' 'laser'
 'laser' 'laser' 'laser' 'laser' 'laser' 'laser' 'laser' 'laser' 'laser'
 'laser' 'laser' 'laser' 'laser' 'laser' 'laser' 'laser' 'laser' 'laser'
 'laser' 'laser' 'laser' 'laser' 'laser' 'laser' 'laser' 'laser' 'laser']
(72,)


(72, 18)

In [63]:
X_orig = np.concatenate((orig_sigma_feat, orig_skew_feat, orig_kurt_feat), axis=1)
orig_label = np.array(['original']*X_orig.shape[0])
print(orig_label)
print(orig_label.shape)
X_orig.shape

['original' 'original' 'original' 'original' 'original' 'original'
 'original' 'original' 'original' 'original' 'original' 'original'
 'original' 'original' 'original' 'original' 'original' 'original'
 'original' 'original' 'original' 'original' 'original' 'original'
 'original' 'original' 'original' 'original' 'original' 'original'
 'original' 'original' 'original' 'original' 'original' 'original'
 'original' 'original' 'original' 'original' 'original' 'original'
 'original' 'original' 'original' 'original' 'original' 'original'
 'original' 'original' 'original' 'original' 'original' 'original'
 'original' 'original' 'original' 'original' 'original' 'original'
 'original' 'original' 'original' 'original' 'original' 'original'
 'original' 'original' 'original' 'original' 'original' 'original'
 'original' 'original']
(74,)


(74, 18)

In [64]:
clf = make_pipeline(StandardScaler(), LinearSVC())

In [65]:
X = np.concatenate((X_laser, X_orig))
X.shape

(146, 18)

In [66]:
y = np.concatenate((laser_label, orig_label))
y.shape

(146,)

In [67]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [68]:
X_train.shape

(97, 18)

In [69]:
clf.fit(X_train, y_train)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('linearsvc', LinearSVC())])

In [70]:
y_pred = clf.predict(X_test)

In [71]:
print("accuracy = ", accuracy_score(y_test, y_pred))

accuracy =  0.9387755102040817


In [72]:
cm = confusion_matrix(y_test, y_pred)

In [73]:
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()

In [74]:
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=clf.classes_)
disp.plot()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [75]:
clf_svc = make_pipeline(StandardScaler(), SVC(gamma='auto'))

In [76]:
clf_svc.fit(X_train, y_train)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('svc', SVC(gamma='auto'))])

In [77]:
y_pred_svc = clf_svc.predict(X_test)

In [78]:
print("accuracy = ", accuracy_score(y_test, y_pred_svc))

accuracy =  0.9795918367346939


In [79]:
cm_svc = confusion_matrix(y_test, y_pred_svc)

In [80]:
disp = ConfusionMatrixDisplay(confusion_matrix=cm_svc, display_labels=clf_svc.classes_)
disp.plot()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Finding the culprit

### For linear svm

In [81]:
idx = y_pred != y_test

In [82]:
idx

array([False, False, False, False, False, False, False, False,  True,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False,  True,
       False, False, False, False, False, False, False, False, False,
       False, False,  True, False, False, False, False, False, False,
       False, False, False, False])

In [83]:
culprits = X_test[idx]
print(culprits)

[[-9.96350670e+00 -9.03244019e+00 -9.75135040e+00 -1.10929441e+01
  -1.23947325e+01 -1.38791943e+01  2.07577258e-01  1.17187932e-01
   5.24207056e-01  7.90367365e-01  9.87694934e-02 -3.77616696e-02
   1.44669922e+00  2.80772567e+00  5.98823880e+00  8.24248656e+00
   8.26030263e+00  9.88947971e+00]
 [-9.97022247e+00 -9.87539196e+00 -1.04409170e+01 -1.16417503e+01
  -1.27828035e+01 -1.40405703e+01  2.65644521e-01 -2.38064341e-02
  -7.23901927e-01  5.59004426e-01  7.96677530e-01 -2.78263912e-02
   1.40464201e+00  6.40608260e+00  1.49161759e+01  1.48904069e+01
   2.17871689e+01  3.05380075e+01]
 [-1.00350142e+01 -9.44056702e+00 -1.01057196e+01 -1.11435738e+01
  -1.26459894e+01 -1.43627567e+01  3.98951203e-01  2.49353349e-01
   4.89179879e-01  3.21609318e-01  4.28350091e-01  3.68442535e-01
   7.76036692e-01  1.70294401e+00  5.48935999e+00  6.86467635e+00
   7.67130485e+00  7.68522881e+00]]


In [92]:
culp1_id = culprits[0] == X_laser
culp1 = X_laser[culp1_id]
print(culp1)

[ -9.9635067   -9.03244019  -9.7513504  -11.09294415 -12.39473248
 -13.87919426   0.20757726   0.11718793   0.52420706   0.79036736
   0.09876949  -0.03776167   1.44669922   2.80772567   5.9882388
   8.24248656   8.26030263   9.88947971]


In [93]:
culp2_id = culprits[1] == X_laser
culp2 = X_laser[culp2_id]
print(culp2)

[-9.97022247e+00 -9.87539196e+00 -1.04409170e+01 -1.16417503e+01
 -1.27828035e+01 -1.40405703e+01  2.65644521e-01 -2.38064341e-02
 -7.23901927e-01  5.59004426e-01  7.96677530e-01 -2.78263912e-02
  1.40464201e+00  6.40608260e+00  1.49161759e+01  1.48904069e+01
  2.17871689e+01  3.05380075e+01]


In [94]:
culp3_id = culprits[2] == X_laser
culp3 = X_laser[culp3_id]
print(culp3)

[-10.03501415  -9.44056702 -10.10571957 -11.14357376 -12.64598942
 -14.36275673   0.3989512    0.24935335   0.48917988   0.32160932
   0.42835009   0.36844254   0.77603669   1.70294401   5.48935999
   6.86467635   7.67130485   7.68522881]


In [95]:
laser_files_vad

['/home/hashim/PHD/audio_data/Laser_Original_New/Laser_cleaned/laser_000_000.mp3',
 '/home/hashim/PHD/audio_data/Laser_Original_New/Laser_cleaned/laser_000_001.mp3',
 '/home/hashim/PHD/audio_data/Laser_Original_New/Laser_cleaned/laser_000_002.mp3',
 '/home/hashim/PHD/audio_data/Laser_Original_New/Laser_cleaned/laser_000_003.mp3',
 '/home/hashim/PHD/audio_data/Laser_Original_New/Laser_cleaned/laser_000_004.mp3',
 '/home/hashim/PHD/audio_data/Laser_Original_New/Laser_cleaned/laser_000_005.mp3',
 '/home/hashim/PHD/audio_data/Laser_Original_New/Laser_cleaned/laser_000_006.mp3',
 '/home/hashim/PHD/audio_data/Laser_Original_New/Laser_cleaned/laser_000_007.mp3',
 '/home/hashim/PHD/audio_data/Laser_Original_New/Laser_cleaned/laser_001_000.mp3',
 '/home/hashim/PHD/audio_data/Laser_Original_New/Laser_cleaned/laser_001_001.mp3',
 '/home/hashim/PHD/audio_data/Laser_Original_New/Laser_cleaned/laser_001_002.mp3',
 '/home/hashim/PHD/audio_data/Laser_Original_New/Laser_cleaned/laser_001_003.mp3',
 '/h

In [96]:
culp1_file = laser_files_vad[culp1_id]
culp1_file

TypeError: only integer scalar arrays can be converted to a scalar index

In [102]:
[i for i, x in enumerate(culp1_id) if x.all()]

[66]

In [103]:
[i for i, x in enumerate(culp2_id) if x.all()]

[64]

In [104]:
[i for i, x in enumerate(culp3_id) if x.all()]

[68]

In [105]:
laser_files_vad[66]

'/home/hashim/PHD/audio_data/Laser_Original_New/Laser_cleaned/laser_010_003.mp3'

In [106]:
laser_files_vad[64]

'/home/hashim/PHD/audio_data/Laser_Original_New/Laser_cleaned/laser_010_001.mp3'

In [107]:
laser_files_vad[68]

'/home/hashim/PHD/audio_data/Laser_Original_New/Laser_cleaned/laser_010_005.mp3'